# Unlocking the Palate - Evaluating Taste Consensus Among Beer Reviewers

---

Group [**BlackAda**](https://en.wikipedia.org/wiki/Blackadder)

> - Ludek Cizinsky ([ludek.cizinsky@epfl.ch](ludek.cizinsky@epfl.ch))
> - Peter Nutter ([peter.nutter@epfl.ch](peter@nutter@epfl.ch))
> - Pierre Lardet ([pierre.lardet@epfl.ch](pierre@lardet@epfl.ch))
> - Christopher Bastin ([christopher.bastin@epfl.ch](christian@bastin@epfl.ch))
> - Mika Senghaas ([mika.senghaas@epfl.ch](mika@senghaas@epfl.ch))

📣 Message for the TA: The initial section of this notebook is straightforward to execute and doesn't need any special preparation. For the `Analysis` segment, please adhere to the instructions in the [REPRODUCE](./REPRODUCE.md) document and ensure all required data is downloaded.

## Introduction

---

Navigating the world of beer reviews can be a daunting task for non-experts.
Beer aficionados often describe brews as having nuanced flavors such as "grassy
notes" and "biscuity/ crackery malt," with hints of "hay." But do these
descriptions reflect the actual tasting experience? Following a
"wisdom-of-the-crowd" approach, a descriptor can be considered meaningful if
many, independent reviewers use similar descriptors for a beer's taste. To
quantify consensus, we use natural language processing techniques to extract
descriptors of a beer's taste and numerically represent these descriptors to
compute similarity or consensus scores. The consensus scores between beer
reviews will unveil whether there is a shared understanding of taste among beer
geeks.


## Dependencies

---

We load the dependencies required for this project to run.


In [ ]:
# Enable continuous module reloading
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard library
import os

# External library
import matplotlib.pyplot as plt
from matplotlib import font_manager

import numpy as np
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import spacy
from spacy.tokens import DocBin 
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.manifold import TSNE
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Custom modules
from src import utils
from src import extractors
from src import embedders
from src.consensus import ConsensusBase, CosineSimilarity, Correlation, KullbackLeiblerDivergence, JensenShannonDivergence
from src import visualise
from src.aggregator import EmbeddingAggregator

And set some global variables.


In [ ]:
colorstyle = "RdBu"
sns.set_style("dark")
sns.set_palette(colorstyle)

# Pandas settings
pd.options.display.max_colwidth = 150

# Load SpaCy model
NLP = spacy.load("en_core_web_sm", disable=["ner", "parser"])
DOC_BIN = DocBin(attrs=["LEMMA", "POS"])
LOAD_DOCS = True

# URL for the full dataset
DATA_URL = "https://drive.google.com/u/2/uc?id=1IqcAJtYrDB1j40rBY5M-PGp6KNX-E3xq&export=download"

# Subsetting options
SUBSET_DEMO = True # To run the demo on the full dataset follow the REPRODUCE.md instructions
SUBSET_ANALYSIS = False 
NUM_SUBSET_DEMO = 10000 # Expects that `process_data.py --limit NUM_SUBSET_SAMPLES`

# Paths
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, "data")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")

# Random seed
SEED = 42
SHUFFLE_REVIEWS = False # Shuffle the embedding order

# Type of metric
METRIC = "cosine"  # cosine, kl, js, correlation

# If this gives you error, then clone the website repo in the same directory as this repo
IMAGES_PATH = os.path.join("..", "ada-2023-project-blackada-webpage", "src", "assets")

# Ensure data directory exists
os.makedirs(DATA_DIR, exist_ok=True)

Setup of seaborn for our datastory:

In [ ]:
font_path = 'font/Poppins-Regular.ttf' 
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

# Light
light_config = {
    "palette": sns.color_palette([
        "#2654E8", # Primary blue
    ]),
    "style": "white",
    "font": prop.get_name(),
    "rc": {
        'axes.facecolor': (0,0,0,0), 
        'figure.facecolor':(0,0,0,0), 
        'axes.spines.left': False, 
        'axes.spines.bottom': False, 
        'axes.spines.right': False, 
        'axes.spines.top': False,
        'axes.titleweight': 'bold',
        'axes.labelpad': 10,  
        'axes.titlepad': 10,  
    },
}

dark_config = {
    "palette": sns.color_palette([
        "#2654E8", # Primary blue
    ]),
    "style": "dark",
    "rc": {
        'axes.facecolor': (0, 0, 0, 0),
        'figure.facecolor': (0, 0, 0, 0),
        'axes.spines.left': False,
        'axes.spines.bottom': False,
        'axes.spines.right': False,
        'axes.spines.top': False,
        'text.color': 'white',
        'xtick.color': 'white',
        'ytick.color': 'white',
        'axes.labelcolor': 'white',
        'axes.titleweight': 'bold',
        'axes.labelpad': 10,  
        'axes.titlepad': 10,  
    },
    "font": prop.get_name(),
}


SEABORN_CONFIGS = [("light", light_config), ("dark", dark_config)]

## Data

---

We will be working with the beer review data from the
[BeerAdvocate](https://www.beeradvocate.com/) platform.


### Data Download

Due to its size (uncompressed 1.6 GB), the dataset is not included in the
repository but must be downloaded. The course staff has provided the data via
Google Drive. On the first run of this notebook, we download the compressed data
file from Google Drive and extract it to the `data` folder. The compressed file
is ~1.5 GB in size.

After extraction and the removal of unnecessary files (archives, ratings file,
...), the data folder should contain the following files: `beers.csv`,
`breweries.csv`, `users.csv`, `reviews.txt`. The total size of the data is ~2.9
GB.

_NB: Data loading takes around **~8min** on the first run. Subsequent runs of
this cell are instant._


In [ ]:
# Download the BeerAdvocate dataset if it doesn't exist
if not utils.raw_data_exists(DATA_DIR):
    utils.download_data(DATA_URL, data_dir=DATA_DIR)
print(f"Raw beer review data downloaded to {DATA_DIR} ✅.")

### Data Processing

We would like to perform a series of pre-processing steps on the raw data to make it more amenable to analysis. These steps include:

- Merge the reviews data with some additional meta-data about the beers, users
  and breweries (e.g. beer style, user location, ...) and collect in a singe
  multi-column DataFrame.
- We cast each column to the correct type, e.g. `date` is converted to a
  `datetime` object.
- We remove any reviews with any missing values (as there are only very few
  where this is the case)
- We drop columns that are not relevant for our analysis
- We rename columns and organise the DataFrame to have a multi-index with the
  reviews for easier access
- We compute `spacy` objects for each review which are crucial for extracting the
  taste descriptors from the reviews. This is a computationally expensive step
  and we therefore only compute the `lemma` and `pos` attributes of each token
  which are the only ones we need for our analysis.
- We add a new column `style` based on the `substyles` in the original data frame to have another way of grouping the beers

_NB: Procesing all `2.5M` reviews into `spacy` objects takes a long time (~2h). Furthermore, the data is very large due to the metadata that spacy stores for each token. We therefore only load a subset of the metadata as specified in the `DocBin` object which we will need for our later analysis - namely the `lemma` and `pos` attributes. This significantly reduces the disk and memory usage._

In [ ]:
# Process the data if it hasn't been processed yet
if not utils.processed_data_exists(PROCESSED_DIR, NUM_SUBSET_DEMO if SUBSET_DEMO else None):
    utils.process_data(DATA_DIR, PROCESSED_DIR, NLP, DOC_BIN, NUM_SUBSET_DEMO if SUBSET_DEMO else None)

print(f"Processed beer review data saved to {PROCESSED_DIR} ✅.")

### Data Loading

Next, we load the data into a Pandas DataFrame. On the first run, we load all
the reviews from the `reviews.txt` file and populate it with some additional
meta-data from the other files. We then save the DataFrame to a `.feather` file
for faster loading in the future. On subsequent runs, we load the DataFrame from
the `.feather` file if it exists.

_NB: Running this cell for the first time reads in all `2.5M` reviews which
takes **~7min**. Subsequent runs should be much faster, taking about **~1min**._


In [ ]:
# Load all reviews and a subset of reviews (10,000)
reviews = utils.load_data(
    PROCESSED_DIR, NLP, LOAD_DOCS, NUM_SUBSET_DEMO if SUBSET_DEMO else None
)
if SHUFFLE_REVIEWS:
    random_indices = np.random.permutation(len(reviews))
    reviews[("review", "text")] = reviews[("review", "text")].values[random_indices]
    reviews[("review", "doc")] = reviews[("review", "doc")].values[random_indices]

msg = "Subset of Data" if SUBSET_DEMO else "Full Data"
print(f"Loaded {len(reviews)} reviews ✅. ({msg})")

### Sanity Checks

During the data loading (`utils.load_data`) we perform some basic data
pre-processing and merging. Specifically, we do the following:


We check that each of these steps is performed correctly and that the data is
consistent.


In [ ]:
# Check that additional information is loaded in the reviews
additional_cols = [("user", "location")]

for col in additional_cols:
    err_msg = f"❌ Additional column {col} not loaded."
    assert col in reviews.columns, err_msg
print(f"✅ Additional columns loaded.")

In [ ]:
# Check that columns have correct type (e.g. review time is a datetime)
example_types = {
    ("review", "date"): "datetime64[ns]",
    ("review", "rating"): "float64",
    ("review", "text"): "object",
}

for col, dtype in example_types.items():
    err_msg = f"❌ Column has type {reviews[col].dtype} but should be {dtype}"
    assert reviews[col].dtype == dtype, err_msg
print(f"✅ All columns have correct type.")

In [ ]:
# Check that there are no missing values (NaNs)
missing_values = reviews.isna().sum()

err_msg = f"❌ There are {missing_values.sum()} missing values in the dataset!"
assert missing_values.sum() == 0, err_msg
print(f"✅ There are no missing values.")

In [ ]:
# Check that index is integer
indices = list(reviews.index) 

err_msg = f"❌ Index is not integer."
assert list(range(len(indices))) == indices, err_msg
print(f"✅ Indices are integers from 0 to {len(indices)-1}.")

### Understanding the Data

Let's explore the data a bit. In this section we will investigate the total
number of reviews and in various sub-groups, as well as understand basic
statistics about the textual reviews.

_Note: We have a full notebook with more detailed EDA of the data in the
[`playground/eda.ipynb`](https://github.com/epfl-ada/ada-2023-project-blackada/blob/main/playground/eda.ipynb)
notebook. In this notebook we focus on the parts of the data exploration that
are important for our project._


In [ ]:
# Show the first 5 rows of the data
reviews.head(3)

We see that all data is in a single data frame with multi-column indexing. Each
row corresponds to a single review of a beer and denotes the user (`user`), beer
(`beer`) and brewery (`brewery`) meta information, as well as the actual review
data (`review`) in separate column indices. For example, we can look at the keys
individually for the first three reviews.


In [ ]:
# Meta-information on beer for first 3 samples
reviews["beer"].head(3)

In [ ]:
# Meta-information on user for first 3 samples
reviews["user"].head(3)

In [ ]:
# Meta-information on brewery for first 3 samples
reviews["brewery"].head(3)

In [ ]:
# Information about review for first 3 samples
reviews["review"].head(3)

As we see, for each review, we have information on the following features:

1. **Review** (`review`): Review Text, Ratings (Appearance, Aroma, Palate,
   Taste, Overall, Rating), Date
2. **User** (`user`): User ID, User Name, #Ratings, #Reviews, Joined Date,
   Location
3. **Beer** (`beer`): Beer ID, Beer Name, Beer Style, Beer Substyle, ABV (Alcohol By Volume),
   #Ratings, #Reviews
4. **Brewery** (`brewery`): Brewery ID, Brewery Name, Location, #Beers


### Groups

In our analysis we want to compute the consensus between the language used in
reviews of a) all beers, b) beers of the same style, c) beers from the same
brewery and, finally, d) invidual beers. The hypothesis is that the
finer-grained the grouping, the higher the consensus between the reviewers.
However, for the analysis to be meaningful we need to ensure that there are
enough reviews in each group. We therefore compute the number of reviews in each
group and plot the distribution of the number of reviews per group.


In [ ]:
unique_beer_styles = reviews.beer["style"].drop_duplicates()
unique_beer_substyles = reviews.beer["substyle"].drop_duplicates()
unique_breweries = reviews.brewery.drop_duplicates()
unique_beers = reviews.beer.drop_duplicates()

print(f"Number of unique beer styles: {len(unique_beer_styles)}")
print(f"Number of unique beer substyles: {len(unique_beer_substyles)}")
print(f"Number of unique breweries: {len(unique_breweries)}")
print(f"Number of unique beers: {len(unique_beers)}")

We show the sorted number of reviews per beer style, brewery and beer below. The
y-axis is log-scaled to better show the distribution and we add a horizontal
line to show the minimum number of reviews to `100` per beer style, brewery and
beer that we require for our further analysis.


In [ ]:
# Minimum number of reviews for a beer style, brewery, or beer to be included in the analysis
MIN_REVIEWS = 5

# Compute the number of reviews for each element in each group
reviews_per_beer_style = (
    reviews.groupby(by=("beer", "style")).size().sort_values(ascending=False)
)
reviews_per_beer_substyle = (
    reviews.groupby(by=("beer", "substyle")).size().sort_values(ascending=False)
)
reviews_per_brewery = (
    reviews.groupby(by=("brewery", "id")).size().sort_values(ascending=False)
)
reviews_per_beer = (
    reviews.groupby(by=("beer", "id")).size().sort_values(ascending=False)
)

# Plot number of reviews per beer style
fig, axs = plt.subplots(ncols=4, figsize=(20, 5))
for ax, reviews_per_group in zip(
    axs, [reviews_per_beer_style, reviews_per_beer_substyle, reviews_per_brewery, reviews_per_beer]
):
    sns.lineplot(x=range(len(reviews_per_group)), y=reviews_per_group.values, ax=ax)
    ax.plot(
        [0, len(reviews_per_group)],
        [MIN_REVIEWS, MIN_REVIEWS],
        linestyle="--",
        color="black",
    )
    a, b = reviews_per_group.index.names[0]
    ax.set(
        title=f"#Reviews per {a.capitalize()} {b.capitalize()}",
        xlabel="Rank",
        ylabel="Counts (Log)",
        yscale="log",
    )

In [ ]:
# Filter out beer styles with less than MIN_REVIEWS reviews
included_beer_styles = reviews_per_beer_style[
    reviews_per_beer_style >= MIN_REVIEWS
].index
included_breweries = reviews_per_brewery[reviews_per_brewery >= MIN_REVIEWS].index
included_beers = reviews_per_beer[reviews_per_beer >= MIN_REVIEWS].index

# Create masks for filtering out beer styles with less than MIN_REVIEWS reviewsk
min_reviews_beer_style_mask = reviews.beer["style"].isin(included_beer_styles)
min_reviews_breweries_mask = reviews.brewery["id"].isin(included_breweries)
min_reviews_beer_mask = reviews.beer["id"].isin(included_beers)

# Filter out reviews for beer styles with less than MIN_REVIEWS reviews
original_reviews = reviews.copy()
reviews = reviews[
    min_reviews_beer_style_mask & min_reviews_breweries_mask & min_reviews_beer_mask
].reset_index()

print(
    f"✅ Filtering done. Reviews after filtering: {len(reviews)} (Removed {len(original_reviews) - len(reviews)} reviews)"
)

### Reviews Statistics

The textual reviews are central to our analysis and we will be using them to
extract the taste descriptors. Let's look at some statistics about the reviews
to ensure that they are of good quality.


In [ ]:
# Let's show some example reviews
pd.DataFrame(reviews.review.head(10)["text"])

We see that this random sample of 10 reviews consists only of reviews that are
very detailed and descriptive about the beer and its taste. This suggests that
the majority of reviews are of good quality and suited for our analysis.
However, we suspect that there might be some meaningless "spam" reviews that may
skew our results. We will investigate this by checking for outliers in the
review length. We use simple proxies for review length, namely the number of
words and characters in the review.


In [ ]:
# Compute character and word lengths of reviews
character_lengths = reviews.review.text.str.len()
word_lengths = reviews.review.text.apply(lambda x: len(x.split()))

# Distribution of the number of ratings/ reviews per user
fig, ax = plt.subplots(ncols=2, figsize=(20, 5))
sns.histplot(x=character_lengths, kde=True, ax=ax[0])
sns.histplot(x=word_lengths, kde=True, ax=ax[1])

character_lengths_stats = character_lengths.describe()
word_lengths_stats = word_lengths.describe()

ax[0].set(
    title="Distribution of Character Lengths in Reviews",
    xlabel="Character Length",
    ylabel="Frequency",
)
ax[1].set(
    title="Distribution of Word Lengths in Reviews",
    xlabel="Word Lengths",
    ylabel="Frequency",
)

# Show summary statistics
pd.DataFrame(
    [character_lengths_stats, word_lengths_stats],
    index=["Character Lengths", "Word Lengths"],
)

We see that most reviews are around **~408 characters** and **~123 words** long.
There is a slight right-skew in the distribution, meaning that there are some
very long reviews. The very short reviews are probably not very helpful for our
analysis as the numeric representation will not be meaningful. Let's look at
those reviews to see if further processing is required.


In [ ]:
# Show the shortest 0.1% of reviews (by character count)
n = int(len(reviews) * 0.001)
character_sorted = list(character_lengths.sort_values().index.values)
shortest_character_length_reviews = reviews.review[
    reviews.index.isin(character_sorted[:n])
]

pd.DataFrame(shortest_character_length_reviews.text)

In [ ]:
# Show the shortest 0.1% of reviews (by word count)
n = int(len(reviews) * 0.001)
words_sorted = list(word_lengths.sort_values().index.values)
shortest_word_length_reviews = reviews.review[reviews.index.isin(
    words_sorted[:n])]

pd.DataFrame(shortest_word_length_reviews.text)

Upon inspecting the shortest reviews, we can see that most of the shortest
reviews by character count are actually regular reviews that are just short.
However, in the reviews with very little words we can see some "spam" reviews
that are not very helpful for our analysis. It is likely that our extractors are
going to struggle with these kinds of reviews. Therefore, we remove all reviews
that have less than `10` words.


In [ ]:
# Remove the shortest reviews by word count from the dataset
MIN_WORDS = 10
not_filtered_review = reviews.copy()
reviews = reviews[word_lengths >= MIN_WORDS]

print(
    f"Removed {(word_lengths < MIN_WORDS).sum()} reviews with less than {MIN_WORDS} words ✅"
)
print(f"Number of reviews: {len(reviews)}")

## Components

---


### Extractors

Before we embed reviews into a numerical representation, we preprocess them
using different **extractors modules**. For this project, we have considered the
following method:

(1) `DummyExtractor`: This is a dummy extractor that does not do any
preprocessing. It simply returns the input text as is.

(2) `LemmaExtractor`: Tokenizes the text and then uses only _lemmas_ of the
extracted tokens. A lemma is the base form of a word. For example, the lemma of
**was** is **be**. Thus, the `LemmaExtractor` might be thougt of as a text
normaliser which maps all tokens to the normalised space.

(3) `AdjectiveExtractor`: As the name suggests, extract tokens which were
classified by `spaCy` as **adjectives**.

(4) `StopWordExtractor`: Removes common words with no semantic value, this implemantaion also uses lemmatization.

In [ ]:
# Define all extractor models
extractor_models: list[extractors.ExtractorBase] = [
    extractors.DummyExtractor(),
    extractors.LemmaExtractor(),
    extractors.AdjectiveExtractor(),
    extractors.StopwordExtractor()
]

We want to understand the behaviour of each of the extractors in detail. To do
this, we process an example review.


In [ ]:
# Define demo review
demo_review = """Pours with a frothy head then settles to a thin head with thin lacing. 
Transparent. Golden to bronze in color. Dry grains. 
Light notes of citrus - orange. Pilsner-esque. Very light malt sweetness - caramel. 
Moves to a dry hoppy-ness. Light bodied. Dry. Somewhat chalky. Meh. 
Just average. Not one I would suggest to a friend, but thank for the organic 
ingredients.
"""

# Preprocess the example with Spacy
processed_demo_review = [NLP(demo_review)]

In [ ]:
# Run the extractors against the example
transformed_all = []
for extractor in extractor_models:
    transformed_example = extractor.transform(processed_demo_review)
    transformed_all.append(transformed_example[0])

Starting with the `DummyExtractor`, we can use it as a reference baseline for
the other two extractors.


In [ ]:
print("DummyExtractor:\n", transformed_all[0].strip())

Let's look at the `LemmaExtractor` next.


In [ ]:
print("LemmaExtractor\n", transformed_all[1].strip())

As the text below shows, `LemmaExtractor` has normalised the words to their base
form (lemma), a couple of examples:

(1) `grains` -> `grain` (get rid of the plural form)

(2) `settles` -> `settle` (remove `s` from the he/she/it form)

(3) `bodied` -> `body` (stem form)

Apart from the lemmatisation, we can also see that how `spaCy` tokenizes the
text. In particular, it treats punctuation marks as separate tokens. For
example, `.` is a separate token.


Now, we run the `AdjectiveExtractor` on the example review.


In [ ]:
print("AdjectiveExtractor\n", transformed_all[2].strip())

Finally, looking at the `AdjectiveExtractor`, we can see that it strips the text
to only adjectives, thereby potentially losing some useful information. On the
other hand, for the purposes of our analysis, this might be in fact useful as we
only want our embeddings be based on the descriptive words related to beer and
avoid the noise.


Finnaly, we we run `StopWordExtractor` with lemmatisation on the example review.

In [ ]:
print("StopwordExtractor\n", transformed_all[3].strip())

The StopwordExtractor eliminates common stopwords and lemmatizes the remaining text. This approach is chosen for our analysis as it filters out irrelevant words while retaining meaningful non-adjective terms, like **caramel**.



Now, we will run the extractors against the reviews and show the word frequency
and the 10 most frequent words for each extractor.


In [ ]:
# We map the list of docs to the list of preprocessed strings
extracted_reviews = [
    extractor.transform(reviews.review.doc) for extractor in extractor_models
]
frequencies = [utils.get_word_frequency(text) for text in extracted_reviews]

# Plot the word frequency of top-10 words for each extractor
fig, axes = plt.subplots(1, 4, figsize=(12, 5))
for ax, freq, extractor in zip(axes, frequencies, extractor_models):
    sns.barplot(x="frequency", y="word", data=freq.head(10), ax=ax)
    ax.set_title(extractor.name)
    ax.set_xlabel("Word Frequency")
    ax.set_ylabel("Word")

plt.tight_layout()

In summary, each of the extractors works as expected. Given the manual
inspection of the extraction process, we hypothesise that the
`StopWordRemover` with lemmatization is the most suitable one for our task because the extracs both adjectives and other nouns that are relevant for a description of a beer.



### Embedders

We need an embedding module to turn the extracted information from the reviews
into a numeric representation. Let's go over how each one works.

- CountEmbeddors uses sklearn's
  [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).
  Count vectorisation simply assigns each word in the vocabulary to a variable
  in the feature vector, and the values are the counts of each word.

- TFIDF is similar to CountVectorizer, but also multiplies by an 'inverse
  document frequency' term. This weights a word in the vocabular by how
  frequently it appears in the corpus. Very common words are penalised, and
  rarer words are given more weight. This also uses sklearn's
  [TFIDFVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).
  This is done using the following formula:

$$w_{i,j} = tf_{i,j} \times \frac{N}{df_i}$$

where $w_{i,j}$ denotes the TFIDF of the $i$ th term in review $j$, $tf_{i,j}$
is the 'term frequency' (the count vectorization) of term $i$ in review $j$, $N$
is the total number of reviews and $df_i$ is the 'document frequency' of term
$i$ i.e. the number of documents in which $i$ appears. This second half of the
equation corresponds to the 'inverse document frequency' (IDF) of TFIDF.

- BERTEmbeddor uses `bert-base-uncased`
  [from HuggingFace](https://huggingface.co/bert-base-uncased). BERT is a
  bidirectional encoder-only transformer. There are many options for extracting
  embeddings from the model since there are 12 layers, and an embbedding for
  each token input. Currently, the implementation takes the penultimate hidden
  state of the model and takes the mean across all tokens in the input (see
  [this guide](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/)).

- SentenceTransformerEmbeddor uses the recommended `all-MiniLM-L6-v2` model from
  the `sentence-transformers`
  [library](<(https://www.sbert.net/docs/pretrained_models.html)>). These models
  take outputs from BERT, conduct pooling similar to above (e.g. by default,
  mean of last layer), and are trained on various sentence-related NLP problems
  using
  [Siamese networks](https://towardsdatascience.com/a-friendly-introduction-to-siamese-networks-85ab17522942).


Let's initialize the embedder models and run them first on some dummy reviews
and then a small sample of the actual reviews.


In [ ]:
# Initalise embedders
embedding_models: list[embedders.EmbedderBase] = [
    embedders.CountEmbedder(),
    embedders.TfidfEmbedder(sparse_output=False),
    embedders.BertEmbedder(),
    embedders.SentenceTransformerEmbedder(),
]

First, we compute the cosine similary between the first sentence and each
subsequent sentence for each of the embedding models.


In [ ]:
# Create a dataframe with some sample reviews
embedder_demo = pd.DataFrame(
    {
        "text": [
            "The beer is nice, with sweet nutty flavours",
            "This is a very different sentence",
            "Not sweet enough. I like my beer sweet. ",
            "Not sweet at all. Terrible beer. ",
            "Not sweet at all. But I like bitter beers so it is a nice beer. ",
            "Piss yellow beer",
            "Sweet beer",
        ]
    }
)

# Initialise results dataframe
results = pd.DataFrame(
    index=[model.name for model in embedding_models],
    columns=[f"Similarity {i}" for i in range(1, len(embedder_demo))],
)

# Compute the similarity between the first and the nth sentence
for i in range(1, len(embedder_demo)):
    for model in embedding_models:
        similarity = utils.compute_similarity(
            model, embedder_demo["text"][0], embedder_demo["text"][i]
        )
        results.loc[model.name, f"Similarity {i}"] = similarity

print("Similarity between first and nth sentence:")
results.head()

Here we can see the obvious pitfall of using CountVectorizer and TFIDF - they
lose all context. Sentences 3 and 4 would ideally have the lowest similairty
with sentence 1 since they are opposite in meaning. However, these samples all
use the same words which Count and TFIDF interpret as therefore being similar.
If, during the pipeline, we were to group beers by some measure that affects
their sweetness, then in order to confirm out hypothesis we would like to see an
increase of similarity inside each group, but we may lower similarity due to
negations.

However, BERT and SentenceTransformers are not necessarily better. The values
are far less interpretable, with sentence embeddor falling for a similar
negation trap since similarities 3 and 4 are higher than 7. Interestingly,
SentenceTransformer was far better than BERT at differentiating between
sentences on different topic matters (similarity 2). BERT's scores are all
broadly similar, and roughly gets the order in line what we might expect, but we
have little faith that this translates any better than sentence transformer to
the real reviews since these little samples play into BERT's context-aware
strengths.

However, for now, we will try to make conclusions using tf-idf. It is the most
interpretable (we can get out the most impactful words at the end), and so long
as there are enough reviews that are long enough, we should see a meaningful
vocabulary emerge. If the tfidf embeddings seem to be limiting us in the future,
we can experiment with other methods.


Now let's try with some real sample reviews.


In [ ]:
# Create a dataframe with some sample reviews
embedder_sample = pd.DataFrame(
    {"text": reviews.sample(4, random_state=0).review.text.values.tolist()}
)

# Initialise results dataframe
results = pd.DataFrame(
    index=[model.name for model in embedding_models],
    columns=[f"Similarity {i}" for i in range(1, len(embedder_sample))],
)

# Compute the similarity between the first and the nth sentence
for i in range(1, len(embedder_sample)):
    for model in embedding_models:
        similarity = utils.compute_similarity(
            model, embedder_sample["text"][0], embedder_sample["text"][i]
        )
        results.loc[model.name, f"Similarity {i}"] = similarity

print("Similarity between first and nth sentence:")
results.head()

Unsurprisinlgy, Count and TFIDF agree on ordering of similarity. However, BERT
and SentenceTransformer disagree both with this ordering and each other. BERT is
the less 'sure', with very high and close values, as in the previous example.

Reading the reviews, it's very hard to define what the ordering _should_ be,
therefore it is hard to define which embedder has done a better job in this
sample. Further investigation will be carried out for P3.


## Analysis

---



Due to the large size of the dataset, we will now load precomputed embeddings of all of the reviews. These embeddings were computed using the `StopWordExtractor` and `TFIDFEmbeddor`. How to get this data is explained in the [Reporduce](./REPRODUCE.md) file.


In [ ]:
# Load all reviews and emebddings
reviews_all = utils.load_data(PROCESSED_DIR, None, False, None)
embeddings_all = utils.load_embeddings(PROCESSED_DIR, None)  # sparse matrix
vocab = utils.load_vocab(PROCESSED_DIR)
assert len(reviews_all) == embeddings_all.shape[0]
print(f"✅ Loaded {len(reviews_all)} reviews and embeddings.")


# Filter out beer styles with less than MIN_REVIEWS reviews
MIN_REVIEWS = 50
MAX_REVIEWS = None
MIN_WORDS = 50
embeddings_all, reviews_all = utils.filter_data(
    embeddings_all, reviews_all, MIN_REVIEWS, MAX_REVIEWS, MIN_WORDS
)  # Add min words filter
assert len(reviews_all) == embeddings_all.shape[0]
print(f"✅ Filtered {len(reviews_all)} reviews and embeddings.")

# Shuffle the embeddings to prove that the effect isn't random
if SHUFFLE_REVIEWS:
    random_indices = np.random.permutation(embeddings_all.shape[0])
    embeddings_all = embeddings_all[random_indices]
    print(f"✅ Shuffled embeddings.")

### Embedding visualisation
Here we chose manually three disinct beer styles that we will use to visualise the embeddings. From each of these styles we picked one of its most popular representatives from at each level of aggregation. 

In [ ]:
# Hand picked styles
styles = ["Specialty Beers", "Stouts and Porters", "Pale Lagers and Pilsners"]

# Representative substyles for each style
substyle = [
    "Fruit / Vegetable Beer",
    "American Double / Imperial Stout",
    "German Pilsener",
]

# Representative beers for each substyle
beers = [
    "Samuel Adams Cherry Wheat",
    "Founders KBS (Kentucky Breakfast Stout)",
    "Prima Pils",
]
groups = [styles, substyle, beers]
aggregators = [("beer", "style"), ("beer", "substyle"), ("beer", "name")]

Let's understand the embeddings a bit better by plotting them in 2D using SVD. Other methods such as t-SNE are too computationally expensive to run on the full dataset but produce nicer plots. `TruncatedSVD` is very fast on sparse matrices, but it only distinguishes clusters along the directions of the highest singular values or their combinations. Therefore, it cannot distinguish many styles or subtle variations within the data.

In [ ]:
# Embed into 2 dimensions
svd = TruncatedSVD(n_components=2, n_iter=10, random_state=42)
reduced_embeddings = svd.fit_transform(embeddings_all)
reduced_embeddings.shape

In [ ]:
# Setup consistent colors for each group
n = len(groups)
colors = sns.color_palette("husl", n_colors=n)
group_colors = [dict(zip(group, colors)) for group in groups]

In [ ]:
fig, axs = plt.subplots(ncols=len(aggregators), figsize=(5 * len(aggregators), 5))
fig.tight_layout(pad=0)

for i, (ax, agg, group) in enumerate(zip(axs, aggregators, groups)):
    subset = reviews_all[reviews_all[agg].isin(group)]

    colors = group_colors[i]
    visualise.embeddings(
        title=f"By {agg[1].capitalize()}",
        embeddings=reduced_embeddings,
        hue=reviews_all[agg],
        subset=subset.index,
        plot_type="kde",
        fill=True,
        alpha=0.5,
        plot_legend=True,
        ax=ax,
        color_palette=colors,
    )

Here we plot the same thing just as a scetter plot 💔

In [ ]:
fig, axs = plt.subplots(ncols=len(aggregators), figsize=(5 * len(aggregators), 5))
fig.tight_layout(pad=0)
for i, (ax, agg, group) in enumerate(zip(axs, aggregators, groups)):
    # Filter out all reviews that are not in the group
    subset = reviews_all[reviews_all[agg].isin(group)]

    # Normalize the number of reviews per group to the smallest group
    min_number_samples = subset.groupby(agg).size().min()
    subset = subset.groupby(agg).sample(n=min_number_samples, random_state=SEED)

    cmap = sns.cubehelix_palette(start=0, light=1, as_cmap=True)
    visualise.embeddings(
        title=f"By {agg[1].capitalize()}",
        embeddings=reduced_embeddings,
        hue=reviews_all[agg],
        subset=subset.index,
        plot_type="scatter",
        # fill=True,
        alpha=0.2,
        color_palette=group_colors[i],
        plot_legend=False,
        ax=ax,
    )

### Consensus
#### Overview
In this section, we initialize the consensus models to quantify the agreement or similarity within groups of data. This is crucial for understanding the patterns and trends in our dataset.

### Understanding the Consensus Metrics for TF-IDF Analysis

#### Overview

In our analysis, we apply various metrics to the rows of a TF-IDF matrix to measure the consensus or similarity within groups. A TF-IDF matrix represents the importance of words in documents within a corpus, making these metrics particularly relevant for text analysis.

#### Description of Metrics

1. **Cosine Similarity**: 
   - Ideal for comparing rows of a TF-IDF matrix, as it measures the cosine of the angle between two vectors. 
   - Effectively assesses similarity in terms of orientation in a multi-dimensional space, making it robust against differences in document length.
   - Ranges from -1 (exactly opposite) to 1 (exactly the same), with 0 indicating independence.

2. **Kullback-Leibler (KL) Divergence**: 
   - Measures the divergence of one probability distribution from another.
   - In the context of TF-IDF, it can highlight how the word distribution of one document diverges from another.
   - **Adjusted for Similarity**: We use $ \frac{1}{1 + KL} $ to quantify similarity, with values closer to 1 indicating more similarity.

3. **Jensen-Shannon (JS) Divergence**: 
   - A symmetric and smoothed variant of KL Divergence.
   - **Adjusted for Similarity**: We apply $\frac{1}{1 + JS} $, where values near 1 suggest high similarity.

4. **Correlation (Pearson)**: 
   - Assesses the linear relationship between two sets of data.
   - In TF-IDF analysis, it helps in understanding the linear association between the term frequencies of different documents.

#### Implications and Performance

- **Cosine Similarity** is highly effective in text analysis, particularly with TF-IDF where it accounts for the relative importance of terms in documents.
- **KL and JS Divergence** provide a deeper insight into how the term distributions vary between documents.
- **Pearson Correlation** offers an understanding of linear relationships in term frequencies.
- Running metrics other than Cosine Similarity can be more computationally intensive but may yield other results than Cosine Similarity that was used for the main analysis. 


In [ ]:
# Initialise the consensus models
consensus: ConsensusBase
if METRIC == "cosine":
    consensus = CosineSimilarity()
elif METRIC == "kl":
    consensus = KullbackLeiblerDivergence()
elif METRIC == "js":
    consensus = JensenShannonDivergence()
elif METRIC == "correlation":
    consensus = Correlation()
else:
    raise ValueError(f"Unknown metric {METRIC}")
print(f"✅ Using {type(consensus).__name__} consensus.")

#### Initializing Aggregators
To facilitate the analysis of consensus within different beer categories, we initialize aggregator classes. These aggregators will help us group the data by different levels such as beer name, style, substyle, and overall general category.

In [ ]:
# Initialise the aggregators
beer_aggregator = EmbeddingAggregator(
    embeddings_all, reviews_all, consensus, ("beer", "name")
)
style_aggregator = EmbeddingAggregator(
    embeddings_all, reviews_all, consensus, ("beer", "style")
)
substyle_aggregator = EmbeddingAggregator(
    embeddings_all, reviews_all, consensus, ("beer", "substyle")
)
gereral_aggregator = EmbeddingAggregator(embeddings_all, reviews_all, consensus, None)

print(f"ℹ️ Beer groups: {len(beer_aggregator.groups)}")
print(f"ℹ️ Style groups: {len(style_aggregator.groups)}")
print(f"ℹ️ Substyle groups: {len(substyle_aggregator.groups)}")
print(f"ℹ️ General groups: {len(gereral_aggregator.groups)}")

Let's perform a detailed statistical analysis on the consensus distributions for each category. Please note, this process is time-intensive and is expected to take approximately `15 minutes`. The analysis proceeds as follows: For each hierarchical level of data aggregation – encompassing the overall beer category, specific beer styles, substyles, and individual beers – we select either the maximum number of samples defined by `MAX_SAMPLES` or the actual number of reviews available for that category. Next, we calculate the consensus matrix for these data points, utilizing the designated `METRIC`. Finally, we determine the average pairwise similarity for these groups and record the results in the consensus_df.

In [ ]:
# Make a nice df
levels = ["General", "Style", "Substyle", "Beer"]
consensus_levels = [
    gereral_aggregator,
    style_aggregator,
    substyle_aggregator,
    beer_aggregator,
]
MAX_SAMPLES = 10000
consensus_df = []
for level, consensus_level in zip(levels, consensus_levels):
    for group in tqdm(consensus_level.groups, desc=f"Processing Group {level}"):
        con = consensus_level.get_consensus_distribution(
            group, max_samples=MAX_SAMPLES
        ).mean()

        consensus_df.append(
            {
                "level": level,
                "group": group,
                "consensus": con,
            }
        )
        dis = consensus_level.get_embeddings_by_group(group).mean(axis=0)

consensus_df = pd.DataFrame(consensus_df)
print(f"️✅ Created consensus dataframe of shape {consensus_df.shape}.")

Here we display the mean average of the individual means of each group. The Number of samples increases with the level of aggregation.

In [ ]:
# Sort by levels list
level_stats = consensus_df.groupby(by=["level"]).describe().reindex(levels, level=0)
level_stats

While the consensus score does not change a lot, the mean consensus across
all groups increases with increasing specificity of the grouping. This is promising
for confirming our hypothesis that language used differs between beer types.


In [ ]:
# Plot distributions of mean consensus scores
fig, axs = plt.subplots(ncols=2, figsize=(20, 6))
fig.suptitle("Distribution Shift", fontsize=16)

# Boxplot of consensus scores per level
sns.boxplot(
    data=consensus_df[consensus_df["level"] != "General"],
    x="level",
    y="consensus",
    hue="level",
    order=levels[1:],
    showfliers=False,
    palette="CMRmap",
    ax=axs[0],
)

# Distplot of consensus scores per level
sns.histplot(
    data=consensus_df[consensus_df["level"] != "General"],
    x="consensus",
    hue="level",
    stat="probability",
    kde=True,
    fill=True,
    common_norm=False,
    palette="CMRmap",
    ax=axs[1],
)

print(f"✅ Plotted distribution of all consensus scores in levels.")

### Statistical Tests

We pefrom a statistical test with the null hypothesis that the average consensus scores in each level of grouping are the same. We use the ANOVA test to test this hypothesis at the 1% significance level. We have different number of samples in each group, so we use the type 2 ANOVA test which is more robust to unequal sample sizes. [Post about differnet types of ANOVA](https://www.r-bloggers.com/2011/03/anova-%e2%80%93-type-iiiiii-ss-explained/)

In [ ]:
# We first need to fit an OLS model
model = ols("consensus ~ C(level)", data=consensus_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
print(
    f"The test statistic is {anova_table.iloc[0]['F']:.2f} and the p-value is {anova_table.iloc[0]['PR(>F)']:.2f}."
)

We see that the p-value is very small, so we reject the null hypothesis and conclude that the average consensus scores in each level of grouping are not the same. This is promising for confirming our hypothesis that language used differs between beer types.

We double check that this insn't a result of the methodology means were computed by re-running the whole notebook with `SHUFFLE=True`. Now we should get a large p value and fail to reject the null hypothesis.

We would also like to know which `Levels` are significantly different from each other. We use Tukey's HSD test to test this hypothesis at the 1% significance level. We see that only the **Beer** to **Substyle** and **Beer** to **Style** comparisons are significantly different. 

In [ ]:
turkey_results = pairwise_tukeyhsd(
    consensus_df["consensus"], consensus_df["level"], alpha=0.01
)
turkey_results.summary()

### Layman's Guide to the Beer Lexicon

---

In [ ]:
for mode, seaborn_cfg in SEABORN_CONFIGS:

    # Set the config for given mode light / dark
    sns.set(**seaborn_cfg)

    # Compute the average
    M = embeddings_all.mean(axis=0)
    average_embedding = np.squeeze(np.asarray(M))

    # Select top K
    top_k = 10
    top_k_embeddings = np.argsort(average_embedding)[::-1][:top_k]
    top_k_features = vocab[top_k_embeddings]

    # Plot
    fig, ax = plt.subplots(figsize=(11, 5))
    sns.barplot(y=top_k_features, x=average_embedding[top_k_embeddings], ax=ax, orient="h")
    ax.set_title(f"Top {top_k} Features across all Reviews")
    ax.set_xlabel("Feature")
    ax.set_ylabel("Average Embedding")
    fig.tight_layout()

    # Save the figure
    save_path = os.path.join(IMAGES_PATH, f"{mode}_top_features.png")
    fig.savefig(save_path, dpi=300)